# Setup

In [1]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [4]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [112]:
import datasets
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer
from tenacity import retry, wait_random_exponential, stop_after_attempt
import os
from openai import OpenAI
import random
import time
from termcolor import colored
import json
from IPython.display import display, Markdown, Latex
import ast

Testing the MMLU dataset before writing a function

In [4]:
dataset = datasets.load_dataset('cais/mmlu','all')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

In [5]:
df = pd.DataFrame(dataset['test'])
df.head()

,question,subject,choices,answer
0,Find the degree for the given field extension ...,abstract_algebra,"[0, 4, 2, 6]",1
1,"Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the i...",abstract_algebra,"[8, 2, 24, 120]",2
2,Find all zeros in the indicated finite field o...,abstract_algebra,"[0, 1, 0,1, 0,4]",3
3,Statement 1 | A factor group of a non-Abelian ...,abstract_algebra,"[True, True, False, False, True, False, False,...",1
4,Find the product of the given polynomials in t...,abstract_algebra,"[2x^2 + 5, 6x^2 + 4x + 6, 0, x^2 + 1]",1


In [6]:


num_questions = len(df)
random_questions = random.sample(range(num_questions), 20)

df_selected = df.iloc[random_questions]
df_selected


,question,subject,choices,answer
1636,During the time an aircraft produces a sonic b...,conceptual_physics,"[breaking the sound barrier, pulling out of a ...",2
12304,A 59-year-old man is brought to the emergency ...,professional_medicine,"[Cardiogenic, Distributive, Hypovolemic, Obstr...",2
14041,The numen of Augustus referred to which of th...,world_religions,"[Divine power, Sexual virility, Military acume...",0
9566,The body mass index (BMI) of vegetarians diffe...,nutrition,"[3 units of BMI, 0.5 units of BMI, 1 unit of B...",2
11524,"A defendant, a 14-year-old male, was tried as ...",professional_law,"[No, because a juvenile may be sentenced to li...",2
5202,Most mnemonic devices (like learning the notes...,high_school_psychology,"[semantic encoding, potentiation, recognition,...",3
13757,The influenza virus is mainly controlled in sp...,virology,"[Hygiene, Vaccination, Antiviral drugs, Humani...",1
6406,The official condemnation by the Catholic Chur...,human_sexuality,"[Pope John II's, Pope Paul VI's, Pope John Pau...",1
12591,__________ is the most frequent cause of many ...,professional_psychology,"[Sexual harassment, Breach of confidentiality,...",2
1553,Let suppose a search box of an application can...,computer_security,"[buffer, external storage, processing power, l...",0


# Subset MMLU

In [84]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def subset_MMLU_test(num=20):
  """
  creates a subset of the MMLU Dataset's test split with the chosen number of questions. Default is 20/
  """
  dataset = datasets.load_dataset('cais/mmlu','all')
  df = pd.DataFrame(dataset['test'])
  num_questions = len(df)
  random_questions = random.sample(range(num_questions), num)
  df_selected = df.iloc[random_questions]
  df_selected.to_csv("MMLU_random_20.csv")
  return df_selected


In [85]:
df=subset_MMLU_test()
df

,question,subject,choices,answer
3806,"In Gideon v. Wainwright, the Supreme Court rul...",high_school_government_and_politics,"[to representation by an attorney, not to incr...",0
7342,"Regarding exons, which, if any, of the followi...",medical_genetics,[Some exons in protein-coding genes consist of...,0
12463,Thomas and Chess’s (1977) “goodness-of-fit” mo...,professional_psychology,[a mismatch between the child’s basic temperam...,0
3297,This question refers to the following informat...,high_school_european_history,"[the role of women in army regiments, the runn...",3
8746,For which of these two scenarios does the main...,moral_scenarios,"[Wrong, Wrong, Wrong, Not wrong, Not wrong, Wr...",3
12086,A police detective received an anonymous call ...,professional_law,"[No, because the detective had probable cause ...",2
12305,A 13-month-old child is brought to the emergen...,professional_medicine,"[Hepatitis, Influenza, Pertussis, Poliomyelitis]",1
227,Where are the vital centres located in the bra...,anatomy,"[midbrain, pons, medulla oblongata, cerebellum]",2
4731,A student eats 3 slices of pizza while studyin...,high_school_microeconomics,"[The student would not eat any more pizza., Th...",3
12614,Which of the following best describes elaborat...,professional_psychology,"[making new information meaningful, consciousl...",0


# Formatting prompts

In [113]:
def formatting_prompt(question, choices, subject):
    prompt = f"""this question is from {subject}
    {question}
    A. {choices[0]}
    B. {choices[1]}
    C. {choices[2]}
    D. {choices[3]}
    Respond with the correct answer from the given choices.\n"""
    return prompt

In [114]:
def create_formatted_prompts(dataframe=None, file_path="MMLU_random_20.csv"):
    formatted_prompts = []

    try:
        # Try reading from the file
        df = pd.read_csv(file_path)
        column_names = df.columns.tolist()

        # Check if the required columns exist in the file
        if 'question' in column_names and 'choices' in column_names and 'subject' in column_names:
            for i in range(len(df)):
                question = df['question'].iloc[i]
                choices = ast.literal_eval(df['choices'].iloc[i])
                subject = df['subject'].iloc[i]
                prompt = formatting_prompt(question, choices, subject)
                formatted_prompts.append(prompt)
        else:
            print(f"Error: One or more required columns ('question', 'choices', 'subject') not found in {file_path}")

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")

        # If the file is not found or doesn't have the required columns, use the provided dataframe
        if dataframe is not None:
            column_names = dataframe.columns.tolist()

            if 'question' in column_names and 'choices' in column_names and 'subject' in column_names:
                for i in range(len(dataframe)):
                    question = dataframe['question'].iloc[i]
                    choices = dataframe['choices'].iloc[i]
                    subject = dataframe['subject'].iloc[i]
                    prompt = formatting_prompt(question, choices, subject)
                    formatted_prompts.append(prompt)
            else:
                print("Error: One or more required columns ('question', 'choices', 'subject') not found in the provided dataframe.")

    messages = []
    for question in formatted_prompts:
        subMessages = []
        subMessages.insert(1, {"role": "user", "content": question})
        messages.append(subMessages)

    return messages

,Unnamed: 0,question,subject,choices,answer
0,2134,Write the prime factorization of 77. Use expon...,elementary_mathematics,"['7 • 11^2 • 13', '7^2 • 11', '7 • 11 • 13', '...",3
1,9877,Nussbaum claims that the specification of part...,philosophy,"['will always consist of a single answer.', 'm...",1
2,7393,In the film 'The Talented Mr Ripley' who plays...,miscellaneous,"['Jude Law', 'Matt Damon', 'Dustin Hoffman', '...",1
3,12951,A person who is low in self-monitoring (Snyder...,professional_psychology,"['his/her life scripts.', 'his/her own feeling...",1
4,11756,A businessman was the owner of an idyllic lake...,professional_law,"['The interest created is a reverter, and the ...",1
5,7481,What does the Latin phrase 'e pluribus unum' m...,miscellaneous,"['What a crazy life', 'In God we trust', 'from...",2
6,3108,A stock solution of 12.0 M sulfuric acid is ma...,high_school_chemistry,"['Add 33.3 mL of water to the flask, and then ...",3
7,568,Hydrogen ions are formed when:,clinical_knowledge,"['glycogen becomes depleted.', 'phosphocreatin...",3
8,11579,A law school advertised in various law journal...,professional_law,['parol evidence is admissible to show that th...,3
9,6741,Which of the following describes the fallacy o...,logical_fallacies,['attacking the character or background of som...,2


In [115]:
prompt_list=create_formatted_prompts(df)
print(prompt_list)

[[{'role': 'user', 'content': 'this question is from elementary_mathematics\n    Write the prime factorization of 77. Use exponents where possible.\n    A. 7 • 11^2 • 13\n    B. 7^2 • 11\n    C. 7 • 11 • 13\n    D. 7 • 11\n    Respond with the correct answer from the given choices.\n'}], [{'role': 'user', 'content': 'this question is from philosophy\n    Nussbaum claims that the specification of particular virtues:\n    A. will always consist of a single answer.\n    B. might sometimes turn out to be a disjunction.\n    C. is necessarily relative to a culture.\n    D. can be settled in such a way as to be no longer revisable.\n    Respond with the correct answer from the given choices.\n'}], [{'role': 'user', 'content': "this question is from miscellaneous\n    In the film 'The Talented Mr Ripley' who plays Mr Ripley?\n    A. Jude Law\n    B. Matt Damon\n    C. Dustin Hoffman\n    D. Ben Affleck\n    Respond with the correct answer from the given choices.\n"}], [{'role': 'user', 'conte

In [62]:
os.environ["OPENAI_API_KEY"] = ""

In [63]:
# Create the OpenAI client
client = OpenAI()

In [14]:
model="gpt-3.5-turbo"

# prompting model and storing their responses

In [18]:
def prompt_model_and_store_response(prompt_list,model):
  responses = []
  for prompt in prompt_list:
    response = client.chat.completions.create(model=model, messages=prompt)
    response_content = response.choices[0].message.content
    print(response_content)
    responses.append(response_content)
  return responses


In [24]:
gpt_35_turbo_responses=prompt_model_and_store_response(prompt_list,model)

B. Title is entirely in the sister as the surviving joint tenant.
A. The length of a football field in yards
D. the engineer.
A. 20
B. A house of worship
A. the state has the burden of proof to show that there is a compelling state interest to support the statute.
A. natural laws.
B. circular reasoning.
A. "unethical because it violates the prohibition against “fee splitting."
D. all of the above
B. 58%
A. ENIAC
A. The resonance pulls Io in different directions and generates heat.
B. DRAM
C. Person-oriented advertising
B. HC2H3O2 and KC2H3O2
A. less mass per nucleon
B. converted into alanine and glutamine and released from the muscle.
A. (∃x)[(Ax • Cx) • (∃y)(Py • Nyx)]
The correct answer is C. 50. 

Since 1 inch on the map represents 25 miles, 5 inches on the map would represent 5 x 25 = 125 miles.


In [25]:
len(gpt_35_turbo_responses)

20

# Formatting prompts for the Judge LLM

In [28]:
def formatting_prompt_judge(question, choices, subject,answer):
    prompt = f"""
    this question is from {subject}
    {question}
    The correct answer is {choices[answer]}
    Check the LLMs response and respond with 0 and 1 for incorrect and correct answers respectively\n"""
    return prompt

In [29]:
def create_formatted_prompts_judge(df,response_list):
  system_message="""
  You are judgeGPT. Your objective is to correctly score the responses of other assistants.
  You would be given the question, the subject it is from, and the correct answer by the user.
  Respond with 0 or 1, to denote wrong answers and correct answers respectively.
  Do NOT respond with anything except 0 or 1.
  You should be ethical and unbiased.
  DO NOT ANSWER THE QUESTION. GRADE THE ANSWER OF THE ASSISTANT
  """
  formatted_prompts = []
  for i in range(len(df)):
      question = df['question'].iloc[i]
      choices = df['choices'].iloc[i]
      subject = df['subject'].iloc[i]
      answer =df['answer'].iloc[i]
      prompt = formatting_prompt_judge(question, choices, subject,answer)
      formatted_prompts.append(prompt)

  messages = []
  for i in range(len(formatted_prompts)):
      subMessages = []
      subMessages.insert(1,{"role": "system", "content": system_message})
      subMessages.insert(2,{"role": "user", "content": formatted_prompts[i]})
      subMessages.insert(3,{"role": "assistant", "content": response_list[i]})
      messages.append(subMessages)
  return messages

In [57]:
test_prompts= create_formatted_prompts_judge(df,gpt_35_turbo_responses)

In [58]:
print(test_prompts)

[[{'role': 'system', 'content': '\n  You are judgeGPT. Your objective is to correctly score the responses of other assistants. \n  You would be given the question, the subject it is from, and the correct answer by the user.\n  Respond with 0 or 1, to denote wrong answers and correct answers respectively. \n  Do NOT respond with anything except 0 or 1.\n  You should be ethical and unbiased.\n  DO NOT ANSWER THE QUESTION. GRADE THE ANSWER OF THE ASSISTANT\n  '}, {'role': 'user', 'content': '\n    this question is from professional_law\n    A brother and a sister purchased land under a deed that conveyed title to them as joint tenants with right ofsurvivorship. Common law joint tenancy is unmodified by statute in the jurisdiction. The purchase price was $50,000, of which the sister paid $10,000 and the brother paid $40,000. The sisterlater mortgaged her interest in the land. The brother then died testate, leaving his entire estate to a cousin. Thesister later paid off her mortgage debt, a

In [32]:
len(test_prompts)

20

# Asking JudgeGPT to evaluate the answers of LLM

In [59]:
def judgements_of_judgeGPT(response_list,model):
  judgement_list = []
  for prompt in response_list:
    judgement = client.chat.completions.create(model=model, messages=prompt)
    response_content = judgement.choices[0].message.content
    print(response_content)
    judgement_list.append(response_content)

  try:
        judgement_list = [int(x) for x in judgement_list]
  except ValueError:
      print("One or more elements in the list could not be converted to integers.")

  return judgement_list


In [60]:
model1="gpt-4-turbo"

In [64]:
gpt_35_turbo_judgments_by_gpt4_Turbo=judgements_of_judgeGPT(test_prompts,model1)

0
0
1
1
0
1
1
0
0
1
0
1
0
1
1
1
0
1
0
0


In [36]:
print(gpt_35_turbo_judgments_by_gpt4_Turbo)

[0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0]


## Adding the total score

In [40]:
def evaluate_results(judgement_list):
  result=0
  for i in judgement_list:
    result+=i
  return result

In [41]:
final_result=evaluate_results(gpt_35_turbo_judgments_by_gpt4_Turbo)

In [42]:
print(final_result)

10


## Everything together to compare three models

Sadly this didn't work. The error is kept here for reference.

In [54]:
#added sleep to get away from rate limit error

def random_dataset_to_prompting_judging_and_evaluating(model1,model2,model3,model4):

  results_list=[]

  df=subset_MMLU_test()
  prompt_list=create_formatted_prompts(df)

  model1_responses=prompt_model_and_store_response(prompt_list,model1)
  time.sleep(15)
  model2_responses=prompt_model_and_store_response(prompt_list,model2)
  time.sleep(15)
  model3_responses=prompt_model_and_store_response(prompt_list,model3)
  time.sleep(15)

  judge_prompts_model1=create_formatted_prompts_judge(df,model1_responses)
  judge_prompts_model2=create_formatted_prompts_judge(df,model2_responses)
  judge_prompts_model3=create_formatted_prompts_judge(df,model3_responses)

  model1_judgements=judgements_of_judgeGPT(judge_prompts_model1,model4)
  time.sleep(15)
  model2_judgements=judgements_of_judgeGPT(judge_prompts_model2,model4)
  time.sleep(15)
  model3_judgements=judgements_of_judgeGPT(judge_prompts_model3,model4)
  time.sleep(15)

  model1_results=evaluate_results(model1_judgements)
  model2_results=evaluate_results(model1_judgements)
  model3_results=evaluate_results(model1_judgements)

  results = [model1_results, model2_results, model3_results]

  model1_results_list = [model1, results[0]]
  model2_results_list = [model2, results[1]]
  model3_results_list = [model3, results[2]]

  results_list.append(model1_results_list)
  results_list.append(model2_results_list)
  results_list.append(model3_results_list)

  return results_list


In [55]:
#keeping gpt-4 as the evaluator and gpt-4o, gpt-3.5-turbo and gpt-4-turbo as the models to be evaluated.
model1="gpt-4-turbo"
model2="gpt-3.5-turbo"
model3="gpt-4o"
model4="gpt-4"

In [56]:
result_list=random_dataset_to_prompting_judging_and_evaluating(model1,model2,model3,model4)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# Configuring the agent

In [155]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [157]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [188]:
judgeGPT_functions = [
    {
        "name": "subset_MMLU_test",
        "description": """Use this function to subset the mmlu""",
        "parameters": {
            "type": "object",
            "properties": {
                "num": {
                    "type": "string",
                    "description": f"""
                            consists of the integer value to select the number of samples from the MMLU. Assume 20 if not mentioned.
                            """,
                }
            },
            "required": ["num"],
        },
    },

    {
        "name": "create_formatted_prompts",
        "description": """Use this function to format the dataset prompts.
        You should NEVER call this function before subset_MMLU_test has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "dataframe": {
                    "type": "object",
                    "description": f"""
                            Dataframe containing the subset MMLU data. Call file 'MMLU_random_20' if name not mentioned.
                            """,
                }
            },
            "required": ["dataframe"],
        },
    },

    {
        "name": "prompt_model_and_store_response",
        "description": """Use this function to get the judgements from judgeGPT.
        You should NEVER call this function before subset_MMLU_test, and create_formatted_prompts has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "model": {
                    "type": "string",
                    "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                    "description": f"""
                            Name of the gpt model to call. Infer this from the user's prompt.
                            """,
                },
                "prompt_list": {
                    "type": "string",
                    "description": f"""
                      The list of formatted prompts. Infer the name of the prompt_list from the user's prompt.
                    """
                }
            },
            "required": ["model", "prompt_list"],
        },
    },

    {
        "name": "create_formatted_prompts_judge",
        "description": """Use this function to format the dataset prompts for judge.
        You should NEVER call this function before subset_MMLU_test has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            Format the prompts of judge before evaluating responses.
                            """,
                }
            },
            "required": ["query"],
        },
    },

    {
        "name": "judgements_of_judgeGPT",
        "description": """Use this function to get the judgements from judgeGPT.
        You should NEVER call this function before subset_MMLU_test, create_formatted_prompts_judge, and create_formatted_prompts has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            Judges the response of the models
                            """,
                }
            },
            "required": ["query"],
        },
    },

    {
        "name": "evaluate_results",
        "description": """Use this function to score the judgements from judgeGPT.
        You should NEVER call this function before subset_MMLU_test, create_formatted_prompts_judge, create_formatted_prompts, and judgements_of_judgeGPT has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "judgement_list": {
                    "type": "string",
                    "description": f"""
                            This is the list of judgements. Infer this from the user.
                            """,
                }
            },
            "required": ["judgement_list"],
        },
    },
]

In [153]:
def call_judgeGPT_functions(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary. Currently extended by adding clauses to this if statement."""
    if full_message.message.function_call.name == "subset_MMLU_test":
        try:
            num = 20  # Default to 20 questions if no value is provided
            print("Generating a subset of questions")
            results = subset_MMLU_test(num)
            try:
                messages.append({
                    "role": "function",
                    "name": full_message.message.function_call.name,
                    "content": str(results),
                })
            except Exception as e:
                print(f"Error appending results: {e}")
            print("Got questions, saving the csv")
            return chat_completion_request(messages)
        except Exception as e:
            print(f"Function execution failed")
            print(f"Error message: {e}")
            messages.append({
                "role": "function",
                "name": full_message.message.function_call.name,
                "content": f"Function execution failed\nError message: {str(e)}",
            })
            return chat_completion_request(messages)

    elif full_message.message.function_call.name == "create_formatted_prompts":
        parsed_output = json.loads(full_message.message.function_call.arguments)
        print("Starting to prompt")
        prompt_list = create_formatted_prompts(parsed_output["prompt_list"])
        return prompt_list

    elif full_message.message.function_call.name == "prompt_model_and_store_response":
        parsed_output = json.loads(full_message.message.function_call.arguments)
        print("Starting to prompt")
        response_list = prompt_model_and_store_response(parsed_output["model", "prompt_list"])
        return response_list

    elif full_message.message.function_call.name == "judgements_of_judgeGPT":
        parsed_output = json.loads(full_message.message.function_call.arguments)
        print("Received responses, starting to judge")
        judgement_list = judgements_of_judgeGPT(parsed_output["query"])
        return judgement_list

    elif full_message.message.function_call.name == "evaluate_results":
        parsed_output = json.loads(full_message.message.function_call.arguments)
        print("Received judgements, starting to score")
        scores_list = evaluate_results(parsed_output["query"])
        return scores_list

    else:
        raise Exception("Function does not exist and cannot be called")

In [99]:
# Start with a system message
paper_system_message = """You are judgeGPT. Your objective is to prompt models with 20 random questions from the MMLU dataset and correctly score the responses of other assistants.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)

# Agent function calling

These are some examples. The function calling was done using the deprecated ```functions``` parameter. Changed later to use the current ```tools``` parameter instead.

In [100]:
paper_conversation.add_message("user", "Hi, create a random subset of MMLU and save it")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=judgeGPT_functions
)
assistant_message = chat_response.choices[0].message.content
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Function generation requested, calling function
Generating a subset of questions
Got questions, saving the csv


Subset of MMLU created!

In [122]:
paper_conversation.add_message(
    "user",
    "Format prompts of the random dataset",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=judgeGPT_functions
)
print(updated_response)

Function generation requested, calling function
Starting to prompt
[[{'role': 'user', 'content': 'this question is from elementary_mathematics\n    Write the prime factorization of 77. Use exponents where possible.\n    A. 7 • 11^2 • 13\n    B. 7^2 • 11\n    C. 7 • 11 • 13\n    D. 7 • 11\n    Respond with the correct answer from the given choices.\n'}], [{'role': 'user', 'content': 'this question is from philosophy\n    Nussbaum claims that the specification of particular virtues:\n    A. will always consist of a single answer.\n    B. might sometimes turn out to be a disjunction.\n    C. is necessarily relative to a culture.\n    D. can be settled in such a way as to be no longer revisable.\n    Respond with the correct answer from the given choices.\n'}], [{'role': 'user', 'content': "this question is from miscellaneous\n    In the film 'The Talented Mr Ripley' who plays Mr Ripley?\n    A. Jude Law\n    B. Matt Damon\n    C. Dustin Hoffman\n    D. Ben Affleck\n    Respond with the co

In [193]:
paper_conversation.add_message(
    "user",
    "Use the prompt_model_and_store_response function. 'model' should go in the model argument and 'updated_response' as prompt_list. Call the prompt_model_and_store_response.",
)
response_list_agent = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=judgeGPT_functions
)
print(response_list_agent)

Unable to generate ChatCompletion response
Exception: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[0].type'.", 'type': 'invalid_request_error', 'param': 'tools[0].type', 'code': 'missing_required_parameter'}}


AttributeError: 'BadRequestError' object has no attribute 'choices'

A few such errors were coming up with the old parameter which I didn't understand, so I went ahead with the new `tools` parameter from here.

In [191]:
tools = [
    {
        "type": "function",
        "function": {
          "name": "subset_MMLU_test",
          "description": """Use this function to subset the mmlu""",
          "parameters": {
              "type": "object",
              "properties": {
                  "num": {
                      "type": "string",
                      "description": f"""
                              consists of the integer value to select the number of samples from the MMLU. Assume 20 if not mentioned.
                              """,
                  }
              },
              "required": ["num"],
          },
      },
    },

    {
        "type": "function",
        "function": {
          "name": "create_formatted_prompts",
          "description": """Use this function to format the dataset prompts.
          You should NEVER call this function before subset_MMLU_test has been called in the conversation.""",
          "parameters": {
              "type": "object",
              "properties": {
                  "dataframe": {
                      "type": "string",
                      "description": f"""
                              Dataframe containing the subset MMLU data. Call file 'MMLU_random_20' if name not mentioned.
                              """,
                  }
              },
              "required": ["dataframe"],
          },
      },
    },

    {
        "type": "function",
        "function": {
          "name": "prompt_model_and_store_response",
          "description": """Use this function to get the judgements from judgeGPT.
          You should NEVER call this function before subset_MMLU_test, and create_formatted_prompts has been called in the conversation.""",
          "parameters": {
              "type": "object",
              "properties": {
                  "model": {
                      "type": "string",
                      "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                      "description": f"""
                              Name of the gpt model to call. Infer this from the user's prompt.
                              """,
                  },
                  "prompt_list": {
                      "type": "string",
                      "description": f"""
                        The list of formatted prompts. Infer the name of the prompt_list from the user's prompt.
                      """
                  }
              },
              "required": ["model", "prompt_list"],
          },
      },
    },

    {
        "type": "function",
        "function": {
          "name": "create_formatted_prompts_judge",
        "description": """Use this function to format the dataset prompts for judge.
        You should NEVER call this function before subset_MMLU_test has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "dataframe": {
                    "type": "string",
                    "description": f"""
                            MMLU subset dataset. Infer this information from the user.
                            """,
                },
                "response_list": {
                    "type": "string",
                    "description": f"""
                            Responses given by the LLM which need to be evaluated.
                            """,
                }
            },
            "required": ["dataframe","response_list"],
          },
      },
    },

    {
        "type": "function",
        "function": {
          "name": "judgements_of_judgeGPT",
          "description": """Use this function to get the judgements from judgeGPT.
          You should NEVER call this function before subset_MMLU_test, create_formatted_prompts_judge, and create_formatted_prompts has been called in the conversation.""",
          "parameters": {
              "type": "object",
              "properties": {
                  "response_list": {
                      "type": "string",
                      "description": f"""
                              the response of the model that needs to be judged.
                              """,
                  },
                  "model": {
                      "type": "string",
                      "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                      "description": f"""
                              Name of the gpt model to call. Infer this from the user's prompt.
                              """,
                  },
              },
              "required": ["response_list"],
          },
      },
    },

    {
        "type": "function",
        "function": {
          "name": "evaluate_results",
        "description": """Use this function to score the judgements from judgeGPT.
        You should NEVER call this function before subset_MMLU_test, create_formatted_prompts_judge, create_formatted_prompts, and judgements_of_judgeGPT has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "judgement_list": {
                    "type": "string",
                    "description": f"""
                            This is the judgement list that needs to be scored. Infer this from the user.
                            """,
                }
            },
            "required": ["judgement_list"],
        },
      },
    },

    {
        "type": "function",
        "function": {
          "name": "random_dataset_to_prompting_judging_and_evaluating",
        "description": """Use this function to do everything from subsetting MMLU to prompting assistants to judging responses and giving the final result.
        You should NEVER call this function before subset_MMLU_test, create_formatted_prompts_judge, create_formatted_prompts, judgements_of_judgeGPT and evaluate_results has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "model1": {
                    "type": "string",
                    "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                    "description": f"""
                            1st model to be evaluated.
                            """,
                },
                "model1": {
                    "type": "string",
                    "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                    "description": f"""
                            2nd model to be evaluated.
                            """,
                },
                "model2": {
                    "type": "string",
                    "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                    "description": f"""
                            1st model to be evaluated.
                            """,
                },
                "model3": {
                    "type": "string",
                    "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                    "description": f"""
                            3rd model to be evaluated.
                            """,
                },
                "model4": {
                    "type": "string",
                    "enum": ["gpt-3.5-turbo", "gpt-4","gpt-4o","gpt-4-turbo"],
                    "description": f"""
                            The judge LLM.
                            """,
                }
            },
            "required": ["judgement_list"],
        },
      },
    },

]

## Examples of assistant understanding which function to call and their arguments

This was very interesting to me as I understood that agentic behaviour is possible to extract. I can make it work if I get access to more openAI credits.

In [179]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Subset the MMLU dataset"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fXhmifIJ3WIykAqSGd1r4ovy', function=Function(arguments='{"num":"20"}', name='subset_MMLU_test'), type='function')])

In [175]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Format the dataframe to send to model. Hint: The dataframe argument is df"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XJgJ87pGIgnWQEYwdhxyWwV1', function=Function(arguments='{"dataframe":"df"}', name='create_formatted_prompts'), type='function')])

In [184]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "prompt gpt-3.5-turbo on list 'updated_response' and store the response"})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "prompt_model_and_store_response"}}
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PeoEK0fAmSzuANb5uRAIBN9L', function=Function(arguments='{"model":"gpt-3.5-turbo","prompt_list":"updated_response"}', name='prompt_model_and_store_response'), type='function')])

In [187]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Format the prompts of the judge. Use the dataframe df and 'gpt_4_responses' as the response_list "})
chat_response = chat_completion_request(
    messages, tools=tools,
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hj4FpQqURLYpbOohDooN7gWF', function=Function(arguments='{"dataframe":"df","response_list":"gpt_4_responses"}', name='create_formatted_prompts_judge'), type='function')])

In [185]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "use gpt-4o to evaluate the responses present in 'list_of_responses'."})
chat_response = chat_completion_request(
    messages, tools=tools,
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_e4WiQW3piJMQ5FzlVaiUaUsc', function=Function(arguments='{"response_list":"list_of_responses","model":"gpt-4o"}', name='judgements_of_judgeGPT'), type='function')])

In [190]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Score the judgements based on judgement_list 'score_gpt4'."})
chat_response = chat_completion_request(
    messages, tools=tools,
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0PcDvwO3O04FgjhpXeQuUmXF', function=Function(arguments='{"judgement_list":"score_gpt4"}', name='evaluate_results'), type='function')])

In [192]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "subset 20 questions and evaluate the performance of gpt-4, gpt-4-turbo, and gpt-3.5 using gpt-4o as the judge."})
chat_response = chat_completion_request(
    messages, tools=tools,
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_a9W23YkgF8n0FLRWqDeJZRsf', function=Function(arguments='{"model1":"gpt-4","model2":"gpt-4-turbo","model3":"gpt-3.5-turbo","model4":"gpt-4o"}', name='random_dataset_to_prompting_judging_and_evaluating'), type='function')])

### That's all that I could do in 7-8 hours. Thank you so much for seeing my project. I had a lot of fun.